In [ ]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD
import re

# Fonction pour parser et générer un fichier RDF
def parse_moves_and_generate_rdf(file_path, output_path):
    # Définir le namespace
    POKE = Namespace("http://example.org/pokemon#")

    # Créer un graphe RDF
    g = Graph()
    g.bind("poke", POKE)

    with open(file_path, 'r', encoding='utf-8') as f:
        data = f.read()

    # Regex pour extraire les mouvements et leurs attributs
    move_pattern = re.compile(r"\|\d+\n\|\{\{m\|(.+?)\}\}\n\{\{typetable\|(.+?)\}\}\n\{\{statustable\|(.+?)\}\}\n\|(\d+|—)\n\|(\d+|∞|—)\n\|(\d+%|—%)")

    for match in move_pattern.finditer(data):
        name, type_, category, pp, power, accuracy = match.groups()

        # Normaliser les valeurs
        name = name.strip()
        type_ = type_.strip()
        category = category.strip()
        pp = None if pp == "—" else int(pp)
        power = None if power in ["—", "∞"] else int(power)
        accuracy = None if accuracy == "—%" else int(accuracy.strip('%'))

        # Ajouter les triplets au graphe RDF
        move_uri = URIRef(POKE[name.replace(' ', '_')])
        g.add((move_uri, RDF.type, POKE.Move))
        g.add((move_uri, RDFS.label, Literal(name, datatype=XSD.string)))
        g.add((move_uri, POKE.type, Literal(type_, datatype=XSD.string)))
        g.add((move_uri, POKE.category, Literal(category, datatype=XSD.string)))
        if pp is not None:
            g.add((move_uri, POKE.pp, Literal(pp, datatype=XSD.integer)))
        if power is not None:
            g.add((move_uri, POKE.power, Literal(power, datatype=XSD.integer)))
        if accuracy is not None:
            g.add((move_uri, POKE.accuracy, Literal(accuracy, datatype=XSD.integer)))

    # Sauvegarder le graphe RDF dans un fichier TTL
    g.serialize(destination=output_path, format="turtle")

# Exemple d'utilisation
parse_moves_and_generate_rdf("moves.txt", "moves_output.ttl")


RDF data has been written to moves_output.ttl.


In [31]:
import re

def generate_moves_with_numbers(input_file, output_file):
    # Open the input file and parse its contents
    with open(input_file, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    rdf_entries = []
    move_number = None
    for line in lines:
        # Match the move number
        match_number = re.match(r'\|\s*(\d+)', line)
        if match_number:
            move_number = match_number.group(1)
            continue

        # Match the move name
        match_move = re.search(r'\{\{m\|([^\}]+)\}\}', line)
        if match_move:
            move_name = match_move.group(1)
            # Replace spaces with underscores and remove single quotes in the move name
            move_uri_name = move_name.replace(" ", "_").replace("'", "")

            rdf_entry = f"""
move:{move_uri_name} a pkmn:Move ;
    rdfs:label "{move_name}" ;
    pkmn:n "{move_number}" ;
    pkmn:blockedByProtect true ;
    pkmn:damagecategory "Special" ;
    pkmn:gameimage "{move_name} IX.png" ;
    pkmn:gameimagewidth "300" ;
    pkmn:gen "I" ;
    owl:sameAs bulbapedia:{move_uri_name} .
"""
            rdf_entries.append(rdf_entry.strip())

    # Write the output file
    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.write("""@prefix move: <http://example.org/pokemon/move/> .
@prefix pkmn: <http://example.org/pokemon/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix bulbapedia: <https://bulbapedia.bulbagarden.net/wiki/> .

""")
        outfile.write("\n\n".join(rdf_entries))

# Update file names as needed
input_file = "moves.txt"
output_file = "moves_output.ttl"

generate_moves_with_numbers(input_file, output_file)


In [2]:
def replace_rdfs_with_schema(input_file, output_file):
    """
    Replace all occurrences of rdfs:label with schema:name in an RDF Turtle file.

    :param input_file: Path to the input Turtle file
    :param output_file: Path to the output Turtle file
    """
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    updated_lines = []
    for line in lines:
        # Replace rdfs:label with schema:name
        updated_line = line.replace("rdfs:label", "schema:name")
        updated_lines.append(updated_line)

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.writelines(updated_lines)


input_file = "moves_output.ttl"
output_file = "moves_output.ttl"
replace_rdfs_with_schema(input_file, output_file)


In [32]:
from pyshacl import validate
from rdflib import Graph

def validate_rdf_with_shacl(rdf_file, shacl_file):
    """
    Validate an RDF file against a SHACL file.

    :param rdf_file: Path to the RDF file to validate.
    :param shacl_file: Path to the SHACL file with constraints.
    :return: Tuple (conforms, report) where conforms is a boolean indicating
             if the RDF conforms to the SHACL constraints, and report is a string report.
    """
    # Load RDF and SHACL files into RDFLib Graphs
    rdf_graph = Graph()
    shacl_graph = Graph()

    rdf_graph.parse(rdf_file, format="turtle")
    shacl_graph.parse(shacl_file, format="turtle")

    # Validate the RDF graph against the SHACL graph
    conforms, report_graph, report_text = validate(
        data_graph=rdf_graph,
        shacl_graph=shacl_graph,
        inference="rdfs",
        abort_on_error=False,
        meta_shacl=False,
        debug=False
    )

    return conforms, report_text

# File paths
rdf_file = "moves_output.ttl"
shacl_file = "shacl.ttl"

# Perform validation
conforms, report = validate_rdf_with_shacl(rdf_file, shacl_file)

# Print results
if conforms:
    print("The RDF file conforms to the SHACL constraints.")
else:
    print("The RDF file does NOT conform to the SHACL constraints.")

print("Validation Report:")
print(report)


Usage of abort_on_error is deprecated. Use abort_on_first instead.


The RDF file conforms to the SHACL constraints.
Validation Report:
Validation Report
Conforms: True

